**注意事項**

このノートブックは、GPU:「T4」に対応させたものです。
「L4」版のノートブックとはモデル等が異なるため、生成される内容が異なることが考えられます。

生成される内容と、ノートブックに記載されている説明が一致しない場合があることをご了承ください。

生成内容とノートブックの説明をよく見比べ、適宜読み替えながら演習を進めてみてください。

---

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/tshigata/lecture-ai-engineering.git

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# HuggingFace Login
from huggingface_hub import notebook_login
from huggingface_hub import login

login(token=hf_token)

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

In [ ]:
# 質問リストを定義
questions = [
    "松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？",
    "未成年者が講座を受講するには、どのような条件が求められますか？",
    "受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？",
    "講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？",
    "講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？"
]

In [ ]:
for question in questions:
  print(question)


# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [ ]:
def llm_answer(question):
    messages = [
        {"role": "user", "content": question},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=False,
    )

    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
# ベースライン回答の生成（RAGなし）
baseline_answers = [llm_answer(q) for q in questions]
for answer in baseline_answers:
  print(answer)

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

# 2. RAGモデル評価

In [ ]:
with open("/content/lecture-ai-engineering/day3/data/松尾・岩澤研究室 講座受講規約.txt", "r", encoding="shift_jis") as f:
    raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[0:2])

In [ ]:
import torch

# GPUのメモリを解放
torch.cuda.empty_cache() #-> NG

# # CPUに切り替え
emb_model = emb_model.to("cpu")

In [ ]:
# 参照テキストのembeddingを生成する
doc_embeddings = emb_model.encode(documents, convert_to_tensor=True)

In [ ]:
def get_references_from_question(question, topk=5):
    q_embedding = emb_model.encode([question], convert_to_tensor=True)
    scores = torch.matmul(q_embedding, doc_embeddings.T)[0].cpu().numpy()
    top_indices = scores.argsort()[::-1][:topk]
    return "\n".join([f"* {documents[i]}" for i in top_indices])

In [ ]:
rag_prompts = [
    f"以下の講座規約を参考にして質問に答えてください。\n\n{get_references_from_question(q)}\n\n質問：{q}"
    for q in questions
]

print(rag_prompts)

In [ ]:
rag_answers = []
for i, q in enumerate(questions):
  rag_prompt = f"以下の講座規約を参考にして質問に答えてください。\n\n{get_references_from_question(q)}\n\n質問：{q}"
  rag_response = llm_answer(rag_prompt)
  rag_answers.append(rag_response)

In [ ]:
# prompt: # questions, baseline_answers, rag_answers を一つのDataframeに収める

import pandas as pd

# Assuming questions, baseline_answers, and rag_answers are already defined as in your provided code

df = pd.DataFrame({
    'questions': questions,
    'baseline_answers': baseline_answers,
    'rag_answers': rag_answers
})

df


In [ ]:
df.to_csv("llm_responses.csv", index=False, encoding="utf-8")

In [ ]:
baseline_answers

In [ ]:
# prompt: questions、baseline_answers、rag_answersを一つのテーブルに変換して、CSVファイルを作成

import pandas as pd

# Assuming questions, baseline_answers, and rag_answers are already defined from the previous code

results = []
for i in range(len(questions)):
    results.append({
        "question": questions[i],
        "baseline_answer": baseline_answers[i],
        "rag_answer": rag_answers[i]
    })

df = pd.DataFrame(results)
df.to_csv("llm_responses.csv", index=False, encoding="utf-8")


# 3. LLM as a Judge

In [ ]:
golden_answers = [
    # Q1: 退会後も研究室が受講者の情報を利用できる条件
    "受講者が退会した後であっても、在籍中に取得された情報はプライバシーポリシーを遵守する形で、本規約に従って研究室が引き続き利用することができます。",  #:contentReference[oaicite:0]{index=0}

    # Q2: 未成年者が講座を受講する条件
    "未成年者が受講を希望する場合は、事前に法定代理人の同意を得ている必要があります。",  #:contentReference[oaicite:1]{index=1}

    # Q3: 他人のSlack投稿をSNSで共有することの取り扱い
    "Slack上でのやりとりやスクリーンショットをSNSにアップすることは禁止されています。",  #:contentReference[oaicite:2]{index=2}

    # Q4: 資料や動画の再利用・再配布の可否
    "講義資料や動画のURLについて、受講後であっても無断での再利用や配布は認められていません。",  #:contentReference[oaicite:3]{index=3}

    # Q5: ノイズなどのトラブル時の対応
    "受講者のインフラに起因するノイズ等のトラブルで他の受講環境に悪影響がある場合、改善するまで参加を一時的に制限されることがあります。"  #:contentReference[oaicite:4]{index=4}
]

In [ ]:
# HTML読み込み（テキストとして評価に含める）
with open("/content/lecture-ai-engineering/day3/data/松尾・岩澤研究室 講座受講規約.html", "r", encoding="utf-8") as f:
    reference_html = f.read()

In [ ]:
template_three_criteria_with_ref = (
    "You are an expert evaluator for answers generated by an AI system.\n"
    "Below is the full official reference document that should be used to evaluate the answers.\n\n"
    "[Reference Document Start]\n{reference_text}\n[Reference Document End]\n\n"
    "Please evaluate the User Answer against this reference document and the question provided using the following criteria:\n"
    "1. Correctness (0-5): Is the User Answer factually correct based on the reference document?\n"
    "2. Completeness (0-5): Does the User Answer cover all necessary points from the reference document?\n"
    "3. Relevance (0-5): Is the User Answer directly relevant to the question?\n\n"
    "Only return your answer in this format:\n"
    "Correctness: <0-5>\nCompleteness: <0-5>\nRelevance: <0-5>\n\n"
    "### Question:\n{query}\n\n### User Answer:\n{user_answer}"
)

In [ ]:
template_strict_eval = (
    "You are an expert evaluator judging whether a User Answer is factually aligned with a given official document.\n\n"
    "The question has only one correct answer, and it must be consistent with the provided reference document.\n"
    "Please strictly evaluate the User Answer according to the three criteria below, using ONLY the reference document.\n\n"
    "If a fact or claim is not explicitly supported by the reference document, deduct points.\n"
    "Even if the User Answer sounds fluent or plausible, deduct points if it includes hallucinated or irrelevant content.\n\n"
    "Criteria:\n"
    "1. Correctness (0-5): Factually correct and consistent with the reference?\n"
    "2. Completeness (0-5): Does it fully answer the question based on the reference?\n"
    "3. Relevance (0-5): Does it directly address the question without extraneous or fabricated content?\n\n"
    "Answer format (numbers only):\n"
    "Correctness: <0-5>\nCompleteness: <0-5>\nRelevance: <0-5>\n\n"
    "### Reference Document:\n{reference_text}\n\n"
    "### Question:\n{query}\n\n"
    "### User Answer:\n{user_answer}"
)


In [ ]:
def evaluate_answer_with_reference(query, user_answer, golden_answer):
    """
    golden_answer（模範解答）を参照して user_answer の正確性・完全性・関連性を評価します。
    OpenAI GPT-4o-miniを使って3軸評価（0-5）を返します。
    """

    # 厳格な評価テンプレート（golden_answerをreferenceとして使う）
    prompt = (
        "あなたはAIによる回答の評価者です。\n\n"
        "以下は「ある質問」に対するユーザーの回答と、それに対応する模範解答（golden answer）です。\n"
        "模範解答と比較して、ユーザーの回答がどの程度正確で、完全で、質問に直接関連しているかを評価してください。\n\n"
        "評価基準は以下の通りです：\n"
        "1. 正確性（Correctness）: golden answerと矛盾なく事実に基づいているか？\n"
        "2. 完全性（Completeness）: golden answerに含まれる要点をどれだけカバーしているか？\n"
        "3. 関連性（Relevance）: 回答が質問に対して直接的か？無関係な話が混じっていないか？\n\n"
        "それぞれ0〜5点で評価し、以下の形式で返してください。\n\n"
        "Correctness: <0-5>\nCompleteness: <0-5>\nRelevance: <0-5>\n\n"
        "### 質問:\n{query}\n\n"
        "### ユーザーの回答:\n{user_answer}\n\n"
        "### 模範解答:\n{golden_answer}\n"
    ).format(query=query, user_answer=user_answer, golden_answer=golden_answer)

    try:
        response = openai_generator(prompt)
        scores = [int(s) for s in re.findall(r"\d+", response)]

        if len(scores) == 3:
            return scores
        else:
            print("スコア抽出失敗:", response)
            return [0, 0, 0]

    except Exception as e:
        print("評価エラー:", e)
        return [0, 0, 0]

In [ ]:
from openai import OpenAI
from google.colab import userdata
import pandas as pd

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

# OpenAI 呼び出し
def openai_generator(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# 評価関数：各項目ごとにスコア抽出
import re

# def evaluate_answer_with_reference(query, user_answer, reference_text):
#     prompt = template_three_criteria_with_ref.format(
#         query=query,
#         user_answer=user_answer,
#         reference_text=reference_text
#     )
#     output = openai_generator(prompt)

#     import re
#     try:
#         scores = [int(s) for s in re.findall(r"\d+", output)]
#         if len(scores) == 3:
#             return scores
#         else:
#             print("スコア抽出失敗:", output)
#             return [0, 0, 0]
#     except:
#         print("評価エラー:", output)
#         return [0, 0, 0]

# 評価実行
records = []

for i in range(len(questions)):
    row = {
        "question": questions[i],
        "baseline_answer": baseline_answers[i],
        "rag_answer": rag_answers[i]
    }

    # golden_answers[i] を参照にして採点
    row["baseline_correctness"], row["baseline_completeness"], row["baseline_relevance"] = evaluate_answer_with_reference(
        questions[i], baseline_answers[i], golden_answers[i]
    )
    row["rag_correctness"], row["rag_completeness"], row["rag_relevance"] = evaluate_answer_with_reference(
        questions[i], rag_answers[i], golden_answers[i]
    )

    records.append(row)

# DataFrameに変換・保存
df_eval = pd.DataFrame(records)
df_eval.to_csv("llm_evaluation_detailed.csv", index=False, encoding="utf-8")
df_eval


In [ ]:
def evaluate_answer_with_reference(query, user_answer, golden_answer):
    """
    golden_answerを基準に user_answer を評価し、各項目のスコアとその理由（説明）を返す。
    戻り値: (correctness, completeness, relevance, explanation)
    """

    prompt = (
        "あなたはAIによる回答の評価者です。\n\n"
        "以下はある質問に対するユーザーの回答と、その模範解答（golden answer）です。\n"
        "以下の3つの観点でユーザー回答を0〜5点で評価し、それぞれについて簡単な理由を説明してください。\n\n"
        "【観点】\n"
        "1. 正確性（Correctness）: golden answerと矛盾せず、事実として正しいか？\n"
        "2. 完全性（Completeness）: golden answerに含まれる要点がカバーされているか？\n"
        "3. 関連性（Relevance）: 回答が質問に直接関連し、無関係な内容を含んでいないか？\n\n"
        "【出力形式】（必ず以下の形式で）\n"
        "Correctness: <スコア>（理由）\n"
        "Completeness: <スコア>（理由）\n"
        "Relevance: <スコア>（理由）\n\n"
        "### 質問:\n{query}\n\n"
        "### ユーザーの回答:\n{user_answer}\n\n"
        "### 模範解答:\n{golden_answer}\n"
    ).format(query=query, user_answer=user_answer, golden_answer=golden_answer)

    try:
        response = openai_generator(prompt)

        # スコアの抽出
        scores = [int(s) for s in re.findall(r"(?<=: )\d", response)]
        explanations = {}
        for line in response.strip().split("\n"):
            if line.startswith("Correctness"):
                explanations["correctness_reason"] = line
            elif line.startswith("Completeness"):
                explanations["completeness_reason"] = line
            elif line.startswith("Relevance"):
                explanations["relevance_reason"] = line

        if len(scores) == 3:
            return (
                scores[0], scores[1], scores[2],
                explanations.get("correctness_reason", ""),
                explanations.get("completeness_reason", ""),
                explanations.get("relevance_reason", "")
            )
        else:
            print("スコア抽出失敗:", response)
            return 0, 0, 0, "N/A", "N/A", "N/A"

    except Exception as e:
        print("評価エラー:", e)
        return 0, 0, 0, "N/A", "N/A", "N/A"


In [ ]:
records = []

for i in range(len(questions)):
    row = {
        "question": questions[i],
        "baseline_answer": baseline_answers[i],
        "rag_answer": rag_answers[i],
        "golden_answer": golden_answers[i]
    }

    # baseline
    b_corr, b_comp, b_rel, b_exp_corr, b_exp_comp, b_exp_rel = evaluate_answer_with_reference(
        questions[i], baseline_answers[i], golden_answers[i]
    )
    row["baseline_correctness"] = b_corr
    row["baseline_completeness"] = b_comp
    row["baseline_relevance"] = b_rel
    row["baseline_correctness_explanation"] = b_exp_corr
    row["baseline_completeness_explanation"] = b_exp_comp
    row["baseline_relevance_explanation"] = b_exp_rel

    # rag
    r_corr, r_comp, r_rel, r_exp_corr, r_exp_comp, r_exp_rel = evaluate_answer_with_reference(
        questions[i], rag_answers[i], golden_answers[i]
    )
    row["rag_correctness"] = r_corr
    row["rag_completeness"] = r_comp
    row["rag_relevance"] = r_rel
    row["rag_correctness_explanation"] = r_exp_corr
    row["rag_completeness_explanation"] = r_exp_comp
    row["rag_relevance_explanation"] = r_exp_rel

    records.append(row)


In [ ]:
# DataFrame に変換
df_eval = pd.DataFrame(records)

In [ ]:
# DataFrame に変換
df_eval = pd.DataFrame(records)

In [ ]:
score_columns = [
    "baseline_correctness", "baseline_completeness", "baseline_relevance",
    "rag_correctness", "rag_completeness", "rag_relevance"
]

# スコア列のみ抽出して表示
df_eval[score_columns]

In [ ]:


# スコア列（数値）のみ削除
score_columns = [
    "baseline_correctness", "baseline_completeness", "baseline_relevance",
    "rag_correctness", "rag_completeness", "rag_relevance"
]
df_eval = df_eval.drop(columns=score_columns, errors="ignore")

# CSV に保存
df_eval.to_csv("llm_evaluation_detailed.csv", index=False, encoding="utf-8-sig")
df_eval

# 4. RAGのデバッグ

In [ ]:
def debug_rag_chunks_for_question(index, questions, documents, doc_embeddings, emb_model, top_k=5):
    """
    質問に対して、類似度の高いチャンクとそのスコアを表示する（FAISS非使用）。

    Parameters:
        index (int): questionsのインデックス
        questions (list of str): 質問リスト
        documents (list of str): 事前に分割された文書チャンク群
        doc_embeddings (Tensor): documentsに対応する埋め込みベクトル（convert_to_tensor=Trueで生成）
        emb_model: SentenceTransformerなどのエンコーダー
        top_k (int): 表示する上位チャンク数
    """
    question = questions[index]
    print(f"\n=== [質問 {index}] ===\n{question}\n")

    # 質問の埋め込みベクトルを生成
    q_embedding = emb_model.encode([question], convert_to_tensor=True)

    # 類似度スコア（内積）を計算
    scores = torch.matmul(q_embedding, doc_embeddings.T)[0].cpu().numpy()

    # 上位 top_k のインデックスとスコア
    top_indices = scores.argsort()[::-1][:top_k]

    print(f"--- 類似チャンク（Top {top_k}） ---")
    for rank, i in enumerate(top_indices):
        print(f"[{rank+1}] 類似度スコア: {scores[i]:.4f}")
        print(f"内容: {documents[i][:300]}...")
        print("-" * 40)

In [ ]:
debug_rag_chunks_for_question(
    index=2,  # questions[2]
    questions=questions,
    documents=documents,
    doc_embeddings=doc_embeddings,
    emb_model=emb_model,
    top_k=5
)

In [ ]:
#

In [ ]:
def identify_factual_errors(query, user_answer, reference_text):
    prompt = (
        "You are a strict factual checker. Given a reference document, a question, and a user answer, "
        "identify any factual inaccuracies in the user answer based on the reference.\n\n"
        "If the user answer contains claims not supported by the reference, or contradicts it, list them.（日本語で）\n"
        "If there are no factual errors, simply say: None.\n\n"
        "### Reference Document:\n{reference_text}\n\n"
        "### Question:\n{query}\n\n"
        "### User Answer:\n{user_answer}\n\n"
        "Factual Errors:"
    ).format(reference_text=reference_text, query=query, user_answer=user_answer)

    return openai_generator(prompt)


In [ ]:
def compare_baseline_vs_rag(query, baseline_answer, rag_answer, reference_text):
    prompt = (
        "You are an impartial evaluator. Based ONLY on the reference document, determine which answer is more accurate and appropriate for the given question.\n\n"
        "Choose the better one strictly based on factual correctness, completeness, and relevance to the question.\n"
        "If both answers are equally good, say: Equal\n"
        "Choose from: Baseline / RAG / Equal\n\n"
        "### Reference Document:\n{reference_text}\n\n"
        "### Question:\n{query}\n\n"
        "### Baseline Answer:\n{baseline_answer}\n\n"
        "### RAG Answer:\n{rag_answer}\n\n"
        "Better Answer:".format(
            reference_text=reference_text,
            query=query,
            baseline_answer=baseline_answer,
            rag_answer=rag_answer
        )
    )

    return openai_generator(prompt)


In [ ]:
records = []

for i in range(len(questions)):
    question = questions[i]
    base_ans = baseline_answers[i]
    rag_ans = rag_answers[i]

    # ベースラインとRAGの事実誤認指摘
    baseline_errors = identify_factual_errors(question, base_ans, reference_html)
    rag_errors = identify_factual_errors(question, rag_ans, reference_html)

    # 相対評価
    comparison_result = compare_baseline_vs_rag(question, base_ans, rag_ans, reference_html)

    records.append({
        "question": question,
        "baseline_answer": base_ans,
        "rag_answer": rag_ans,
        "baseline_errors": baseline_errors,
        "rag_errors": rag_errors,
        "better_answer": comparison_result.strip()
    })

df_compare = pd.DataFrame(records)
df_compare.to_csv("rag_vs_baseline_comparison.csv", index=False, encoding="utf-8")
df_compare
